In [ ]:
import requests
import pandas as pd
import json
import plotly.express as px
from plotly.subplots import make_subplots


# Obtenemos las claves de acceso a la API
url = "https://api.invertironline.com/token"

payload = 'username={tu_usuario}&password={tu_contraseña}*&grant_type=password'
headers = {
  'Content-Type': 'application/x-www-form-urlencoded',
  'Cookie': '1ea603=90N+IRNhzS+ZAlji2JeWwwFHYqnB5ZldfmpPr25mmXPfJx3JyDKWDLm2ANNR0cnLgvJJGe0qOY6g/L/4ppP/uvsPRie6OCjdqveFKwQxSr1ezBsEsymWYUkPcg9hDa8WUnpyEbGvdOsRU3DCvgzlkClGmeRHinBuMlRNLwRye7fv0V3+'
}

response = requests.request("POST", url, headers=headers, data=payload)
# Convertimos la respuesta a JSON
response_json = response.json()

# KEYS
bearer_token = response_json["access_token"]
refresh_token = response_json["refresh_token"]

In [3]:
# ---------------   PANEL ACCIONES ARGENTINAS  ----------------
List_panel_Acciones = ["Merval", "Panel General", "Merval 25", "Merval Argentina", "Burcap", "CEDEARs"]


panel_data = []

def obtener_panel(instrumento, panel, pais):
    
    url_panel2 = f"https://api.invertironline.com/api/Cotizaciones/{instrumento}/{panel}/{pais}"
    
    payload = {}
    headers = {
    'Authorization': f"Bearer {bearer_token}",
    'Cookie': 'i18n.langtag=es-AR; isMobile=0; 1ea603=2qWeR/Ko7fTv5qFwUUmajFxWKNKtMyB66jJlJwlyFgldqvfrivSOqffj72nSSQ8p18sxZLeNp+m5CDB+KmkD7BxBPsZzBlzrTNpk728p80GSm7j9dYAIvHBuX7tgmx5HVqjmBADYnrBeAlt9fjpu+dxROhkucWhAX8YWLux7ISiIj1gU'
    }

    response = requests.request("GET", url_panel2, headers=headers, data=payload)
    panel_data.append(response.json())

obtener_panel("Acciones", List_panel_Acciones[0], "argentina")

# panel_data_json = json.dumps(panel_data, indent=2)

ticket = []
volumen = []
precio_maximo = []
precio_minimo = []
ultimo_precio = []
var_porcentual = []

for titulo in panel_data[0]["titulos"]:
    
    ticket_t = titulo["simbolo"]
    ticket.append(ticket_t)
     
    volumen_t = titulo["volumen"]
    volumen.append(volumen_t)
    
    precio_maximo_t = titulo["maximo"]
    precio_maximo.append(precio_maximo_t)
    
    precio_minimo_t = titulo["minimo"]
    precio_minimo.append(precio_minimo_t)
    
    ultimo_precio_t = titulo["ultimoCierre"]
    ultimo_precio.append(ultimo_precio_t)
    
    var_porcentual_t = titulo["variacionPorcentual"]
    var_porcentual.append(var_porcentual_t)

data_relevante = pd.DataFrame({
    "ticket" : ticket, 
    "volumen" : volumen,
    "precio_maximo" : precio_maximo,
    "precio_minimo" : precio_minimo,
    "ultimo_precio" : ultimo_precio,
    "var_porcentual" : var_porcentual
})

data_relevante = data_relevante.sort_values(by="var_porcentual", ascending=False)

# Agregamos una columna para diferencias variaciones positivas y negativas
data_relevante['variación'] = data_relevante['var_porcentual'].apply(lambda x: 'Positivo' if x > 0 else 'Negativo')

# Grafico de plotly
var_merval = px.bar(
    data_relevante, 
    x="ticket", 
    y="var_porcentual", 
    color="variación",  # Color por valor de 'var_porcentual'
    title="Variación diaria de acciones en el Merval",
    color_discrete_map={'Positivo': 'green', 'Negativo': 'red'})

#creamos grafico de lineas para mostrar el volumen operado durante la jornada
vol_merval = px.line(data_relevante, 
                     x="ticket", 
                     y="volumen", 
                     title="Volumen operado por instrumento")

# Crear subplots y aplicar un tema
fig_informe = make_subplots(rows=1, cols=2, 
                            subplot_titles=("Variación % Merval", "Volumen operado por acción"),
                            horizontal_spacing=0.15 #especificamos espacio entre columnas del subplot
                            )

# Agregar los gráficos px a los subplots
for var in var_merval.data:
    fig_informe.add_trace(var, row=1, col=1)
for vol in vol_merval.data:
    fig_informe.add_trace(vol, row=1, col=2)
    
# Personalizar layout general del subplot
fig_informe.update_layout(title_text="Informe de Acciones Merval - Variación y Volumen",
    showlegend=True,
    height=500,
    template="plotly_dark")

# Configuración de los ejes x para mejorar la legibilidad
fig_informe.update_xaxes(
    tickangle=-50,  # Girar etiquetas
    row=1, col=1,    # Aplicar en el primer gráfico
    showgrid=False)

fig_informe.update_xaxes(
    tickangle=-50,  # Girar etiquetas
    row=1, col=2,    # Aplicar en el segundo gráfico
    showgrid=False)

# Configuración de los ejes y
fig_informe.update_yaxes(title_text="Variación (%)", row=1, col=1)

fig_informe.update_yaxes(title_text="Volumen Operado", row=1, col=2)

fig_informe.show()

In [4]:
# ---------------   PANEL BONOS ARGENTINOS  ----------------
List_panel_Bonos =["Soberanos en pesos más Cer","Soberanos en pesos a tasa variable", "Soberanos en pesos a tasa fija",
                      "Soberanos en dólares", "Soberanos dolar linked", "Provinciales en pesos", "Provinciales dolar linked", 
                      "Provinciales en dólares", "Provinciales en euros", "Cupones vinculados al PBI", "Bonos corporativos en pesos",
                      "Bonos corporativos en dólares"]

panel_bonos_data = []

def obtener_panel_bonos(instrumento, panel, pais):
    
    url_panel2 = f"https://api.invertironline.com/api/Cotizaciones/{instrumento}/{panel}/{pais}"
    
    payload = {}
    headers = {
    'Authorization': f"Bearer {bearer_token}",
    'Cookie': 'i18n.langtag=es-AR; isMobile=0; 1ea603=2qWeR/Ko7fTv5qFwUUmajFxWKNKtMyB66jJlJwlyFgldqvfrivSOqffj72nSSQ8p18sxZLeNp+m5CDB+KmkD7BxBPsZzBlzrTNpk728p80GSm7j9dYAIvHBuX7tgmx5HVqjmBADYnrBeAlt9fjpu+dxROhkucWhAX8YWLux7ISiIj1gU'
    }

    response = requests.request("GET", url_panel2, headers=headers, data=payload)
    panel_bonos_data.append(response.json())

for descripcion in List_panel_Bonos:
    obtener_panel_bonos("Bonos", descripcion, "argentina")

ticket = []
descripcion = []
volumen = []
precio_maximo = []
precio_minimo = []
ultimo_precio = []
var_porcentual = []

for index, block in enumerate(panel_bonos_data): # ENUMERATE: Permite utilizar los indices de la lista para iterar y obtener los valores de la lista.
    for titulo in block["titulos"]:
        try:
            ticket_t = titulo["simbolo"]
            ticket.append(ticket_t)
            
            descripcion_b = List_panel_Bonos[index] 
            descripcion.append(descripcion_b)
            
            volumen_t = titulo["volumen"]
            volumen.append(volumen_t)
            
            precio_maximo_t = titulo["maximo"]
            precio_maximo.append(precio_maximo_t)
            
            precio_minimo_t = titulo["minimo"]
            precio_minimo.append(precio_minimo_t)
            
            ultimo_precio_t = titulo["ultimoCierre"]
            ultimo_precio.append(ultimo_precio_t)
            
            var_porcentual_t = titulo["variacionPorcentual"]
            var_porcentual.append(var_porcentual_t)
        except Exception:
            pass

data_bonos = pd.DataFrame({
    "ticket" : ticket, 
    "volumen" : volumen,
    "descripcion" : descripcion,
    "precio_maximo" : precio_maximo,
    "precio_minimo" : precio_minimo,
    "ultimo_precio" : ultimo_precio,
    "var_porcentual" : var_porcentual})

data_bonos["variación"] = ["positiva" if value > 0 else "negativa" for value in var_porcentual]
data_bonos = data_bonos.sort_values(by="var_porcentual", ascending=False)

var_bonos = px.bar(data_bonos,
                   x="ticket", y="var_porcentual",
                   color="variación",
                   color_discrete_map={'positiva': 'green', 'negativa': 'red'})
vol_bonos = px.bar(data_bonos,
                   x="ticket", y="volumen",
                   title="Volumen diario - Bonos")

# Crear subplots
fig_informe = make_subplots(rows=1, cols=2, 
                            subplot_titles=("Variación % diaria", "Volumen diario"),
                            horizontal_spacing=0.15)

for var in var_bonos.data:
    fig_informe.add_trace(var, row=1, col=1)
for vol in vol_bonos.data:
    fig_informe.add_trace(vol, row=1, col=2)
    
# Personalizar layout del subplot
fig_informe.update_layout(
    title={"text":"Informe diario de bonos - Variación y Volumen",
           'x': 0.5,
           'xanchor': 'center'},
    showlegend=True,
    height=500,
    width=900,
    template="plotly_dark")

# Configuración de los ejes x
fig_informe.update_xaxes(tickangle=-50, row=1, col=1, showgrid=False)
fig_informe.update_xaxes(tickangle=-50, row=1, col=2, showgrid=False)

# Configuración de los ejes y
fig_informe.update_yaxes(title_text="Variación (%)", row=1, col=1)
fig_informe.update_yaxes(title_text="Volumen Operado", row=1, col=2)

fig_informe.show()